In [ ]:
!nvidia-smi 

Fri Oct 14 01:25:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ACDC'
%run Unet_2D.ipynb
%run '/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ACDC/Unet_2D (copy).ipynb'
%run preprocessing_2D.ipynb
%run otherUnets.ipynb

Given Array is
[-2, 7, 15, -14, 0, 15, 0, 7, -7, -4, -13, 5, 8, -14, 12]
After Sorting Array is
[-14, -14, -13, -7, -4, -2, 0, 0, 5, 7, 7, 8, 12, 15, 15]


In [ ]:
!pip install pytorch-lightning
!pip install timm
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import timm.optim
import torch.nn as nn
from timm.data.loader import create_loader
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_acdc_AC.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_diceRV_epoch", "train_diceMYO_epoch",
                    "train_diceLV_epoch", "val_loss","val_diceRV", "val_diceMYO", "val_diceLV"
                    ]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
data = np.load("./dataACDCA/ACDC_train_aug160.npz")
x_train, y_train = data["image"], data["mask"]
print(data["image"].shape)
print(data["mask"].shape)
data = np.load("./dataACDCA/ACDC_val160.npz")
x_val, y_val = data["image"], data["mask"]
data = np.load("./dataACDCA/ACDC_test160.npz")
x_test, y_test = data["image"], data["mask"]
#x_test = np.concatenate([x_test, x_val], axis = 0)
#y_test = np.concatenate([y_test, y_val], axis = 0)
del data

(5248, 160, 160)
(5248, 160, 160)


In [ ]:
columns = ['0','1','2','3']
df = pd.DataFrame(columns=columns)
_, counts =  np.unique(y_train, return_counts=True)
conts_dict = dict(zip(columns, counts))
df = df.append(conts_dict, ignore_index=True)
label_0 = df['0'].sum()
label_1 = df['1'].sum()
label_2 = df['2'].sum()
label_3 = df['3'].sum()
total_labels = label_0 + label_1 + label_2 + label_3
n_classes = 4
#Class weights claculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels/(n_classes*label_0)), 2)
wt1 = round((total_labels/(n_classes*label_1)), 2)
wt2 = round((total_labels/(n_classes*label_2)), 2)
wt3 = round((total_labels/(n_classes*label_3)), 2)
print("\n Weights are:", wt0, wt1, wt2, wt3)
# 0.27 10.0 8.91 8.94


 Weights are: 0.27 10.0 8.91 8.94


In [ ]:
train_dataset = DataLoader(ACDCLoader(x_train, y_train, transform=False), batch_size=16, pin_memory=True,
                        shuffle=True, num_workers=2, 
                        drop_last=True, prefetch_factor = 16)
val_dataset = DataLoader(ACDCLoader(x_val, y_val, typeData="test"), batch_size=64,
                          num_workers=2, prefetch_factor=64)
test_dataset = DataLoader(ACDCLoader(x_test, y_test, typeData="test"), batch_size=64,
                          num_workers=2, prefetch_factor=64)

In [ ]:
class SemiActiveLoss(nn.Module):
    def __init__(self, device, alpha =1e-9, beta = 1e-1, lamda = 1e-3):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.beta = beta      
        self.lamda = lamda
    def LevelsetLoss(self, image, y_pred, kernel_size=5, smooth=1e-5):
        kernel = torch.ones(1, y_pred.size(1), kernel_size, kernel_size, device=self.device) / kernel_size**2
        padding = kernel_size //2
        lossRegion = 0.0
        y_pred_fuzzy = y_pred ** 2
        for ich in range(image.size(1)):
            target_ = image[:,ich:ich+1] 
            pcentroid_local = F.conv2d(target_ * y_pred_fuzzy + smooth, kernel, padding = padding) \
                                / F.conv2d(y_pred_fuzzy + smooth, kernel, padding = padding)
            plevel_local = target_ - pcentroid_local
            loss_local = plevel_local * plevel_local * y_pred_fuzzy

            pcentroid_global = torch.sum(target_ * y_pred_fuzzy, dim=(2,3),keepdim=True) \
                                / torch.sum(y_pred_fuzzy+smooth, dim=(2,3),keepdim = True)   
            plevel_global = target_ - pcentroid_global
            loss_global = plevel_global * plevel_global * y_pred_fuzzy

            lossRegion += torch.sum(loss_local) + self.beta * torch.sum(loss_global)
        return lossRegion 
    def GradientLoss(self, y_pred, penalty = "l1"):
        dH = torch.abs(y_pred[...,1:] - y_pred[...,:-1])
        dW = torch.abs(y_pred[:,:,1:] - y_pred[:,:,:-1])
        if penalty == "l2":
            dH = dH * dH
            dW = dW * dW
        loss =  torch.sum(dH) +  torch.sum(dW)
        return loss
    def ActiveContourLoss(self, y_true, y_pred, smooth=1e-6):   
        dim = (1,2,3)
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:,1:]
        y_pred = y_pred[:,1:]

        active = - torch.log(1-y_pred+smooth) * (1-yTrueOnehot) - torch.log(y_pred+smooth) * yTrueOnehot
        loss = torch.sum(active, dim = dim) / torch.sum(yTrueOnehot + y_pred - yTrueOnehot * y_pred +smooth, dim = dim)
        return torch.mean(loss)

    def forward(self, image, y_true, y_pred):
        active = self.ActiveContourLoss(y_true, y_pred)
        # levelset =  self.LevelsetLoss(image, y_pred)
        # length = self.GradientLoss(y_pred)
        return active
        # return active + self.alpha * (levelset + self.lamda * length) 

class FocalLoss(nn.Module):
    def __init__(self, device, gamma=2):
        super().__init__()
        self.device = device
        self.gamma = gamma
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred,min=1e-5,max=1-1e-5)

        loss = torch.sum( - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred))
        return loss / (y_true.size(0) * y_true.size(2) * y_true.size(3))

class TverskyLoss(nn.Module):
    def __init__(self, device, alpha=0.7):
        super().__init__()
        self.device = device
        self.alpha = alpha
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = torch.clamp(y_pred, min=1e-5, max=1-1e-5)[:,1:]

        TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
        FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])


        loss = 1-torch.mean((TP + 1e-5) / (TP + self.alpha * FP + (1-self.alpha) * FN + 1e-5))
        return loss

In [ ]:
class ActiveFocalLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS, gamma=2):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
        self.gamma = gamma
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)

        active_focal = - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred) \
                        - (1 - yTrueOnehot) * y_pred**self.gamma * torch.log(1 - y_pred)
        loss = torch.sum(active_focal, dim=[2, 3]) * self.class_weight
        return torch.sum(loss) / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

class ActiveContourLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)

        # y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)
        active = torch.sum(yTrueOnehot * (1 - y_pred) + (1 - yTrueOnehot) * y_pred, dim=[2, 3])
        loss = torch.sum(active * self.class_weight)
        return loss / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

em oi cross entropy thi em dung lenh nay:
```
loss = CrossEntropy(self.device)(y_true, y_pred)
```
ACDC thi ko co diceloss, ngoai ra em chay lenh nay de so sanh voi bai nay cua anh ne: "Semi-supervised Deep
Learning-Based Approach with Multiphase Active Contour Loss for Left Ventricle Segmentation from CMR Images”
```
loss = ActiveContourLoss(self.device)(y_true, y_pred)
```
TH3: em so sanh voi focal loss (anh nghi la ko can so sanh voi cross entropy dau, vi cai focal loss nay la ban moi hon roi, co ma tuy em :)) )
```
loss = FocalLoss(self.device)(y_true, y_pred)
```
TH4: em thich thi lam cai tversky loss, em thich de alpha = bn cung dc :D
```
loss = TverskyLoss(self.device, alpha=0.7)(y_true, y_pred)
```


In [ ]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = PiDiNet(img_channel=1, inplane=32, num_classes=4, dil=4, sa=False, ta=True)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items
    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = CrossEntropy(device=self.device)( y_true, y_pred)
    #     diceRV, diceMYO, diceLV = dice_rv(y_true, y_pred), dice_myo(y_true, y_pred), dice_lv(y_true, y_pred)
    #     return loss, diceRV, diceMYO, diceLV
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        #loss = TverskyLoss(self.device, alpha=0.7)(y_true, y_pred)
        loss_focal = ActiveFocalLoss(self.device, [0.27, 10.00, 8.91, 8.94])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.27, 10.00, 8.91, 8.94])(y_true, y_pred)
        loss = loss_focal + loss_contour
        diceRV, diceMYO, diceLV = dice_rv(y_true, y_pred), dice_myo(y_true, y_pred), dice_lv(y_true, y_pred)
        jacRV, jacMYO, jacLV = jac_rv(y_true, y_pred), jac_myo(y_true, y_pred), jac_lv(y_true, y_pred)
        return loss, diceRV, diceMYO, diceLV, jacRV, jacMYO, jacLV

    def training_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV, jacRV, jacMYO, jacLV = self._step(batch)
        metrics = {"loss": loss, "train_diceRV": diceRV, "train_diceMYO": diceMYO, "train_diceLV": diceLV, "train_jacRV": jacRV, "train_jacMYO": jacMYO, "train_jacLV": jacLV}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV, jacRV, jacMYO, jacLV = self._step(batch)
        metrics = {"val_loss": loss, "val_diceRV": diceRV, "val_diceMYO": diceMYO, "val_diceLV": diceLV, "val_jacRV": jacRV, "val_jacMYO": jacMYO, "val_jacLV": jacLV}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV, jacRV, jacMYO, jacLV = self._step(batch)
        metrics = {"loss":loss, "test_diceRV": diceRV, "test_diceMYO": diceMYO, "test_diceLV": diceLV, "test_jacRV": jacRV, "test_jacMYO": jacMYO, "test_jacLV": jacLV}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=12, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_diceRV"}
        return [optimizer], lr_schedulers
    

initialization done


In [ ]:
segmentor = Segmentor(PiDiNet(img_channel=1, inplane=32, num_classes=4, dil=4, sa=False, ta=True)) #Segmentor(U_Net(drop_prob=0))
os.makedirs('/content/weights/', exist_ok = True) 
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("/content/weights/", filename="ckpt{val_diceRV:0.4f}",
                                                            monitor="val_diceRV", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar()
logger = HistoryLogger()
#swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=25)
PARAMS = {"gpus":1, "benchmark": True, "enable_progress_bar" : True, "logger":False,
        #   "callbacks" : [progress_bar], 
        #    "overfit_batches" :1, 
          "callbacks" : [check_point, progress_bar, logger], 
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":200,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)

# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/best_acdc_160.ckpt")
#segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/current.ckpt")

initialization done


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type    | Params
----------------------------------
0 | model | PiDiNet | 10.4 M
----------------------------------
10.4 M    Trainable params
0         Non-trainable params
10.4 M    Total params
20.851    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:454: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  self.padding, self.dilation, self.groups)


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 328: 'val_diceRV' reached 0.54043 (best 0.54043), saving model to '/content/weights/ckpt0.5404.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 89, global step 29520: 'val_diceRV' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 90, global step 29848: 'val_diceRV' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 91, global step 30176: 'val_diceRV' was not in top 1
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.save_checkpoint("/content/weights/ckpt0.8087.ckpt", weights_only=True)

In [ ]:
'''for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        layer.drop_prob = 0.05
        
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        print(layer.drop_prob) 

# segmentor.configure_optimizers()'''

'for layer in segmentor.modules():\n    if isinstance(layer, DropBlock2D):\n        layer.drop_prob = 0.05\n        \nfor layer in segmentor.modules():\n    if isinstance(layer, DropBlock2D):\n        print(layer.drop_prob) \n\n# segmentor.configure_optimizers()'

In [ ]:
trainer.test(segmentor, test_dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          loss              0.9242124557495117
       test_diceLV          0.8910897374153137
      test_diceMYO          0.8229514956474304
       test_diceRV           0.799781858921051
       test_jacLV           0.8514348268508911
       test_jacMYO          0.7378208637237549
       test_jacRV           0.7405893206596375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': 0.9242124557495117,
  'test_diceRV': 0.799781858921051,
  'test_diceMYO': 0.8229514956474304,
  'test_diceLV': 0.8910897374153137,
  'test_jacRV': 0.7405893206596375,
  'test_jacMYO': 0.7378208637237549,
  'test_jacLV': 0.8514348268508911}]

In [ ]:
'''dice_scores = trainer.test(segmentor, val_dataset)[0]
dice_ave = np.mean(list(dice_scores.values())[1:])
dice_ave'''

'dice_scores = trainer.test(segmentor, val_dataset)[0]\ndice_ave = np.mean(list(dice_scores.values())[1:])\ndice_ave'

In [ ]:
weight_path = sorted(glob.glob("/content/weights/*"), reverse=True)
max_score = 0
best_weight = ""
for weight in tqdm(weight_path):
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
    dice_scores = trainer.test(segmentor, val_dataset)[0]
    dice_ave = np.mean(list(dice_scores.values()))
    clear_output()
    print(dice_ave)
    if max_score < dice_ave:
        max_score = dice_ave
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


100%|██████████| 5/5 [00:06<00:00,  1.22s/it]

0.6919132620096207
best weight is: /content/weights/ckpt0.8673.ckpt with 0.6970881521701813


In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
# x_inf = np.zeros((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2]))
# for i in range(x_test.shape[0]):
#     x_inf[i, 0] = min_max_preprocess(x_test[i])
# segmentor = segmentor.to(device)
# segmentor.eval()
# y_pred = predict(x_inf, segmentor)
# torch.cuda.empty_cache()
# mask_predict = np.argmax(y_pred, axis=1)


In [ ]:
def dice_volume_rv(y_true, y_pred, smooth = 1e-5):
    y_pred = torch.where(y_pred == 1, 1, 0)
    y_true = torch.where(y_true == 1, 1, 0)
    intersection = torch.sum(y_true * y_pred)
    cardinality  = torch.sum(y_true + y_pred)
    return (2. * intersection + smooth) / (cardinality + smooth)

def dice_volume_myo(y_true, y_pred, smooth = 1e-5):
    y_pred = torch.where(y_pred == 2, 1, 0)
    y_true = torch.where(y_true == 2, 1, 0)
    intersection = torch.sum(y_true * y_pred)
    cardinality  = torch.sum(y_true + y_pred)
    return (2. * intersection + smooth) / (cardinality + smooth)

def dice_volume_lv(y_true, y_pred, smooth = 1e-5):
    y_pred = torch.where(y_pred == 3, 1, 0)
    y_true = torch.where(y_true == 3, 1, 0)
    intersection = torch.sum(y_true * y_pred)
    cardinality  = torch.sum(y_true + y_pred)
    return (2. * intersection + smooth) / (cardinality + smooth)

def predict_volume(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = PiDiNet(img_channel=1, inplane=32, dil=16, sa=False, ta=True)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     diceRV, diceMYO, diceLV = dice_rv(y_true, y_pred), dice_myo(y_true, y_pred), dice_lv(y_true, y_pred)
    #     return loss, diceRV, diceMYO, diceLV
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        #loss = CrossEntropy(device=self.device)( y_true, y_pred)
        loss_focal = ActiveFocalLoss(self.device, [0.27, 10.00, 8.91, 8.94])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.27, 10.00, 8.91, 8.94])(y_true, y_pred)
        loss = loss_focal + loss_contour
        diceRV, diceMYO, diceLV = dice_rv(y_true, y_pred), dice_myo(y_true, y_pred), dice_lv(y_true, y_pred)
        return loss, diceRV, diceMYO, diceLV

    def training_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV = self._step(batch)
        metrics = {"loss": loss, "train_diceRV": diceRV, "train_diceMYO": diceMYO, "train_diceLV": diceLV}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV = self._step(batch)
        metrics = {"val_loss": loss, "val_diceRV": diceRV, "val_diceMYO": diceMYO, "val_diceLV": diceLV}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, diceRV, diceMYO, diceLV = self._step(batch)
        metrics = {"test_diceRV": diceRV, "test_diceMYO": diceMYO, "test_diceLV": diceLV}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=15, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_diceRV"}
        return [optimizer], lr_schedulers
    

segmentor = Segmentor(PiDiNet(img_channel=1, inplane=32, dil=16, sa=False, ta=True))


initialization done
initialization done


In [ ]:
#weight_path = sorted(glob.glob("content/weights/*"), reverse=True)
#weight_path[2:3]

In [ ]:
all_files = sorted(glob.glob("./dataACDCA/training/*"))
np.random.seed(42)
np.random.shuffle(all_files)
train_count = 70
train_files = all_files[:train_count]
test_files = all_files[train_count : train_count + 10]+all_files[-10:]
valid_files = all_files[train_count + 10: train_count + 20]
x_inf = np.zeros((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2]))
for i in range(x_test.shape[0]):
    x_inf[i, 0] = min_max_preprocess(x_test[i])

weight_path = sorted(glob.glob("/content/weights/*"), reverse=True)
max_score = 0
best_weight = ""
weight_path = ["/content/weights/ckpt0.8721.ckpt"]
for weight in tqdm(weight_path):
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
    segmentor = segmentor.to(device)
    segmentor.eval()
    y_pred = predict_volume(x_inf, segmentor)
    torch.cuda.empty_cache()
    mask_predict = np.argmax(y_pred, axis=1)
    count = 0
    list_rv, list_myo, list_lv = [], [], []
    for files in test_files:
        list_image = [x for x in glob.glob(files+"/*") if x.find('frame') != -1 and x.find('gt') == -1]
        for image_name in list_image:
            num = image_name.find("nii")
            mask_name = image_name[:num-1] +"_gt.nii.gz" 
            image = nib.load(image_name).get_fdata().astype(np.uint16)
            label = nib.load(mask_name).get_fdata().astype(np.uint8)
            image = center_crop(image)
            label = center_crop(label)
            label_pred = np.zeros_like(label)
            for z in range(label.shape[-1]):
                label_pred[..., z] = mask_predict[count]
                count += 1
            label = torch.from_numpy(label)
            label_pred = torch.from_numpy(label_pred)
            list_rv.append(dice_volume_rv(label, label_pred).item())
            list_myo.append(dice_volume_myo(label, label_pred).item())
            list_lv.append(dice_volume_lv(label, label_pred).item())
    diceRV = np.mean(list_rv)
    diceMYO = np.mean(list_myo)
    diceLV = np.mean(list_lv)
    dice_ave = np.mean([diceRV, diceMYO, diceLV])
    clear_output()
    print(dice_ave)
    if max_score < dice_ave:
        max_score = dice_ave
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


100%|██████████| 1/1 [01:02<00:00, 62.12s/it]

0.9232681587338448
best weight is: /content/weights/ckpt0.8721.ckpt with 0.9232681587338448


In [ ]:
print(diceRV, diceMYO, diceLV, np.mean([diceRV, diceMYO, diceLV]))

0.9162792056798935 0.9014659836888314 0.9520592868328095 0.9232681587338448


In [ ]:
# all_files = sorted(glob.glob("./dataACDCA/training/*"))
# np.random.seed(42)
# np.random.shuffle(all_files)
# train_count = 70
# train_files = all_files[:train_count]
# test_files = all_files[train_count : train_count + 10]+all_files[-10:]
# valid_files = all_files[train_count + 10: train_count + 20]
# x_inf = np.zeros((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2]))
# for i in range(x_test.shape[0]):
#     x_inf[i, 0] = min_max_preprocess(x_test[i])
# weight_path = sorted(glob.glob("./weights/*"), reverse=True)
# # weight_path = ["./weights/ckpt0.8533.ckpt", "./weights/ckpt0.8288.ckpt"]
# max_score = 0
# best_weight = ""
# for weight in tqdm(weight_path):
#     segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
#     segmentor = segmentor.to(device)
#     segmentor.eval()
#     y_pred = predict_volume(x_inf, segmentor)
#     torch.cuda.empty_cache()
#     mask_predict = np.argmax(y_pred, axis=1)
#     count = 0
#     dice_rv_all, dice_myo_all, dice_lv_all = [], [], []
#     for files in test_files:
#         list_rv, list_myo, list_lv = [], [], []
#         list_image = [x for x in glob.glob(files+"/*") if x.find('frame') != -1 and x.find('gt') == -1]
#         for image_name in list_image:
#             num = image_name.find("nii")
#             mask_name = image_name[:num-1] +"_gt.nii.gz" 
#             image = nib.load(image_name).get_fdata().astype(np.uint16)
#             label = nib.load(mask_name).get_fdata().astype(np.uint8)
#             image = center_crop(image)
#             label = center_crop(label)
#             for z in range(label.shape[-1]):
#                 sub_label = torch.from_numpy(label[...,z])
#                 label_pred = torch.from_numpy(mask_predict[count])
#                 list_rv.append(dice_volume_rv(sub_label, label_pred).item())
#                 list_myo.append(dice_volume_myo(sub_label, label_pred).item())
#                 list_lv.append(dice_volume_lv(sub_label, label_pred).item())
#                 count += 1
#             dice_rv_all.append(np.mean(list_rv))
#             dice_myo_all.append(np.mean(list_myo))
#             dice_lv_all.append(np.mean(list_lv))
#     dice_RV = np.mean(dice_rv_all)
#     dice_MYO = np.mean(dice_myo_all)
#     dice_LV = np.mean(dice_lv_all)
#     dice_ave = np.mean([dice_RV, dice_MYO, dice_LV])
#     print(dice_ave)




In [ ]:
np.savez_compressed("acdc_AttUnet_ourLoss", predicted_mask=mask_predict)

In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1)
    plt.subplot(131), plt.imshow(x_test[i])
    plt.subplot(132), plt.imshow(y_test[i])
    plt.subplot(133), plt.imshow(mask_predict[i])
